In [5]:
import numpy as np
import sympy as sp
from sympy import symbols, pprint

# Estimate of `C=O` k and `C=C` K spring Constants

In [ ]:


"""
structure: a string like 'O=C=O'

please don't leave spaces! For example, avoid 'O= C=O'
please note that this is not eaxctly the chemical structure for example
chemical strcture for C3O2 is O=C=C=C=O but we will use 'O=C-C-C=O' to indicate that the bonds are different O=C and C-C

mass_dict: {'O': m, 'C': M} 
spring_dict: {'=': k, '-': K} 
"""
        
import sympy as sp

class SymbolicMolecularChain:
    def __init__(self, structure, mass_dict, spring_dict):
        self.structure = structure
        self.mass_dict = mass_dict
        self.spring_dict = spring_dict
        self.atoms, self.bonds = self._parse_structure(structure)
        self.N = len(self.atoms)

    def _parse_structure(self, structure):
        atoms, bonds = [], []
        i = 0
        while i < len(structure):
            if structure[i].isalpha():
                atoms.append(structure[i])
                if i + 1 < len(structure) and structure[i + 1] in self.spring_dict:
                    bonds.append(structure[i + 1])
                    i += 2
                else:
                    i += 1
            else:
                i += 1
        return atoms, bonds

    def build_matrix(self):
        m = [self.mass_dict[a] for a in self.atoms]
        k = [self.spring_dict[b] for b in self.bonds]

        A = sp.zeros(self.N)
        for i in range(self.N):
            if i > 0:
                A[i, i - 1] = -k[i - 1] / m[i]
                A[i, i] += k[i - 1] / m[i]
            if i < self.N - 1:
                A[i, i + 1] = -k[i] / m[i]
                A[i, i] += k[i] / m[i]
        return A



In [ ]:
class SymbolicEigenSolver:
    def __init__(self, matrix):
        self.matrix = matrix
        self.N = matrix.shape[0]
        self.lambda_sym = sp.Symbol('lambda')

    def characteristic_polynomial(self):
        return (self.matrix - self.lambda_sym * sp.eye(self.N)).det()

    def eigenvalues(self):
        return sp.solve(self.characteristic_polynomial(), self.lambda_sym)

    def diagonalize(self):
        return self.matrix.diagonalize()


In [15]:

m, M, k, K = symbols("m M k K")

CO2 = SymbolicMolecularChain("O=C=O", {'O': m, 'C': M}, {'=': k,'-':K})

Matrix_EignValue_CO2 = CO2.build_matrix()



In [19]:

solver_CO2 = SymbolicEigenSolver(Matrix_EignValue_CO2)
eigenvals = solver_CO2.eigenvalues()
pprint(eigenvals)

⎡   k  M⋅k + 2⋅k⋅m⎤
⎢0, ─, ───────────⎥
⎣   m      M⋅m    ⎦


## Numerical Estimation based on 
http://www2.ess.ucla.edu/~schauble/MoleculeHTML/CO2_html/CO2_page.html

https://orgchemboulder.com/Spectroscopy/irtutor/alkenesir.shtml

In [30]:

c_m_s = 2.99792458e8 

# Atomic masses in kg
u = 1.66053906660e-27
m = 15.999 * u
M = 12.011 * u


In [32]:

#transitional mode
w_1=0

#mid_static_mode
w_2=135360*c_m_s*2*np.pi

#anti_mode
w_3=239630*c_m_s*2*np.pi


k_from_w2 = w_2**2 * m
k_from_w3 = w_3**2 / (1/m + 2/M)
k_from_w2, k_from_w3


(1727.1249969385876, 1477.2810284076643)

In [33]:

k_avg=(k_from_w2+ k_from_w3)/2
k_avg




1602.203012673126

In [35]:

m, M, k, K = symbols("m M k K")

C2 = SymbolicMolecularChain("C=C", {'O': m, 'C': M}, {'=': k,'-':K})

Matrix_EignValue_C2 = C2.build_matrix()


solver_C2 = SymbolicEigenSolver(Matrix_EignValue_C2)
eigenvals = solver_C2.eigenvalues()
pprint(eigenvals)

⎡   2⋅k⎤
⎢0, ───⎥
⎣    M ⎦


In [38]:
M = 12.011 * u

w_cc_1=164000
w_cc_2=168000

w_cc=(w_cc_1+w_cc_2)/2
w_cc= w_cc*c_m_s*2*np.pi


(w_cc**2)* M/2

975.0246320574357

In [ ]:
#K is stiffer than k, so our estimates are consistent. Also they are consistent with those sources! 


# C3O2


In [50]:
'''based on the estimates we have so far I will use 

'''

u = 1.66053906660e-27

m = 15.999 * u
M = 12.011 * u
k=1602.20
K=975.02 


In [51]:

# now I will use numpy



class MolecularChainNumeric:
    def __init__(self, structure, mass_dict, spring_dict):
        self.structure = structure
        self.mass_dict = mass_dict
        self.spring_dict = spring_dict
        self.atoms, self.bonds = self._parse_structure(structure)
        self.N = len(self.atoms)

    def _parse_structure(self, structure):
        atoms, bonds = [], []
        i = 0
        while i < len(structure):
            if structure[i].isalpha():
                atoms.append(structure[i])
                if i + 1 < len(structure) and structure[i + 1] in self.spring_dict:
                    bonds.append(structure[i + 1])
                    i += 2
                else:
                    i += 1
            else:
                i += 1
        return atoms, bonds

    def build_matrix(self):
        m = np.array([self.mass_dict[a] for a in self.atoms])
        k = np.array([self.spring_dict[b] for b in self.bonds])

        A = np.zeros((self.N, self.N))
        for i in range(self.N):
            if i > 0:
                A[i, i - 1] = -k[i - 1] / m[i]
                A[i, i] += k[i - 1] / m[i]
            if i < self.N - 1:
                A[i, i + 1] = -k[i] / m[i]
                A[i, i] += k[i] / m[i]
        return A


class NumericEigenSolver:
    def __init__(self, matrix):
        self.matrix = matrix

    def solve(self):
        # eigenvalues are ω², eigenvectors are mode shapes
        eigenvals, eigenvecs = np.linalg.eigh(self.matrix)
        freqs = np.sqrt(np.clip(eigenvals, 0, None))  # ω = sqrt(λ)
        return freqs, eigenvecs


In [59]:

C3O2 = MolecularChainNumeric("O=C-C-C=O", {'O': m, 'C': M}, {'=': k,'-': K})
A = C3O2.build_matrix()

solver = NumericEigenSolver(A)
frequencies, modes = solver.solve()

for i, (freq, mode) in enumerate(zip(np.sqrt(frequencies), modes.T), start=1):
    print(f"Frequency {i}: {freq:.3e} Hz")
    print(f"Mode {i}: {np.round(mode, 4)} m\n\n")




Frequency 1: 0.000e+00 Hz
Mode 1: [0.677  0.5444 0.3743 0.237  0.2213] m


Frequency 2: 1.189e+07 Hz
Mode 2: [-0.3713 -0.1865  0.1933  0.4942  0.7387] m


Frequency 3: 1.685e+07 Hz
Mode 3: [ 0.399  -0.1013 -0.7562 -0.1629  0.4819] m


Frequency 4: 2.037e+07 Hz
Mode 4: [ 0.3434 -0.4775 -0.1464  0.6998 -0.3779] m


Frequency 5: 2.137e+07 Hz
Mode 5: [ 0.3558 -0.6561  0.4789 -0.4281  0.1743] m





# C9O3


In [60]:

C9O3 = MolecularChainNumeric("O=C-C-C-C=O=C-C-C-C-C=O", {'O': m, 'C': M}, {'=': k,'-': K})
A = C9O3.build_matrix()

solver = NumericEigenSolver(A)
frequencies, modes = solver.solve()

for i, (freq, mode) in enumerate(zip(np.sqrt(frequencies), modes.T), start=1):
    print(f"Frequency {i}: {freq:.3e} Hz")
    print(f"Mode {i}: {np.round(mode, 4)} m\n\n")

Frequency 1: 0.000e+00 Hz
Mode 1: [-0.6465 -0.524  -0.3741 -0.2609 -0.1733 -0.1737 -0.141  -0.1012 -0.0713
 -0.0484 -0.0303 -0.0281] m


Frequency 2: 7.462e+06 Hz
Mode 2: [ 0.2573  0.1832  0.0499 -0.0866 -0.2176 -0.3848 -0.3996 -0.3985 -0.3722
 -0.3223 -0.2519 -0.2656] m


Frequency 3: 1.077e+07 Hz
Mode 3: [ 0.1747  0.1019 -0.0457 -0.1808 -0.2662 -0.3644 -0.2863 -0.0792  0.1497
  0.3374  0.4322  0.5563] m


Frequency 4: 1.354e+07 Hz
Mode 4: [-0.2579 -0.0857  0.2561  0.4219  0.2977  0.13   -0.0827 -0.3753 -0.41
 -0.1628  0.1962  0.4431] m


Frequency 5: 1.494e+07 Hz
Mode 5: [ 0.2926  0.0383 -0.4186 -0.4491 -0.0222  0.3349  0.3119 -0.0436 -0.3546
 -0.3045  0.0558  0.3203] m


Frequency 6: 1.710e+07 Hz
Mode 6: [ 0.1682 -0.0528 -0.3236 -0.0283  0.3165  0.2522 -0.1274 -0.5284 -0.005
  0.5271  0.1371 -0.3279] m


Frequency 7: 1.836e+07 Hz
Mode 7: [ 0.273  -0.1812 -0.5064  0.3455  0.3943 -0.1781 -0.3115  0.1934  0.2488
 -0.2741 -0.1599  0.1808] m


Frequency 8: 1.955e+07 Hz
Mode 8: [-0.1578  